<a href="https://colab.research.google.com/github/Quinera/KokomeloTalk/blob/inoue%2Fdevelop/backend/music_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Spotify APIの認証情報
CLIENT_ID = 'c7fd38830113450392d3560d74cba87a'
CLIENT_SECRET = '8753ea68cde2472e8042f43810086eaf'

# Spotifyオブジェクトの作成（Client Credentials Flow）
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

def search_tracks(query_word, limit=50):
    """
    指定したクエリで曲を検索し、曲IDと詳細情報を取得する。
    ページネーションを利用して複数回のリクエストを行い、指定した総件数を取得する。

    Args:
        query_word (str): 検索クエリ。
        limit (int): 取得する曲の総数（最大100など）。

    Returns:
        list of dict: 曲の詳細情報を含む辞書のリスト。
    """
    max_limit_per_request = 50  # Spotify APIの1リクエストあたりの最大取得件数
    total_tracks = []
    total_fetched = 0

    while total_fetched < limit:
        current_limit = min(max_limit_per_request, limit - total_fetched)
        offset = total_fetched

        query = query_word
        try:
            results = sp.search(q=query, limit=current_limit, type='track', market='JP', offset=offset)
            tracks = results['tracks']['items']

            if not tracks:
                # 取得できる曲がこれ以上ない場合
                break

            for track in tracks:
                track_info = {
                    'id': track['id'],
                    'name': track['name'],
                    'artists': ', '.join([artist['name'] for artist in track['artists']]),
                    'album': track['album']['name'],
                    'release_date': track['album']['release_date'],
                    'duration_ms': track['duration_ms'],
                    'external_url': track['external_urls']['spotify'],
                    'popularity': track['popularity']  # popularityを追加
                }
                total_tracks.append(track_info)

            fetched = len(tracks)
            total_fetched += fetched

            if fetched < current_limit:
                # 取得できた曲がリクエスト数より少ない場合、これ以上結果はない
                break

            time.sleep(0.2)  # 200ミリ秒待機

        except spotipy.exceptions.SpotifyException as e:
            print(f"Spotify APIエラー (limit={current_limit}, offset={offset}): {e}")
            break
        except Exception as e:
            print(f"予期せぬエラーが発生しました (limit={current_limit}, offset={offset}): {e}")
            break

    return total_tracks

def display_tracks(track_info_list):
    """
    曲の詳細情報を表示する。

    Args:
        track_info_list (list of dict): 曲の詳細情報を含む辞書のリスト。
    """
    attr_list = ["id", "name", "artists", "release_date", "danceability", "valence"]

    if not track_info_list:
        print("曲が見つかりませんでした。")
        return

    print(f"見つかった曲の数: {len(track_info_list)}\n")
    for idx, track in enumerate(track_info_list, start=1):
        print(f"曲 {idx}:")
        for attr in attr_list:
            print(f"  {attr}: {track[attr]}")

def get_audio_features(track_ids):
    """
    指定したトラックIDのオーディオ特徴を取得する。

    Args:
        track_ids (list of str): トラックIDのリスト。

    Returns:
        dict: トラックIDをキーとしたオーディオ特徴の辞書。
    """
    audio_features_list = sp.audio_features(track_ids)
    audio_features = {}
    for feature in audio_features_list:
        if feature:  # featureがNoneでない場合
            audio_features[feature['id']] = feature
    return audio_features

def merge_track_info_with_audio_features(track_info_list, audio_features):
    """
    曲の基本情報とオーディオ特徴を統合する。

    Args:
        track_info_list (list of dict): 曲の基本情報のリスト。
        audio_features (dict): トラックIDをキーとしたオーディオ特徴の辞書。

    Returns:
        list of dict: 統合された曲の詳細情報を含む辞書のリスト。
    """
    for track in track_info_list:
        track_id = track['id']
        if track_id in audio_features:
            track.update({
                'acousticness': audio_features[track_id]['acousticness'],
                'danceability': audio_features[track_id]['danceability'],
                'energy': audio_features[track_id]['energy'],
                'instrumentalness': audio_features[track_id]['instrumentalness'],
                'key': audio_features[track_id]['key'],
                'liveness': audio_features[track_id]['liveness'],
                'loudness': audio_features[track_id]['loudness'],
                'mode': audio_features[track_id]['mode'],
                'speechiness': audio_features[track_id]['speechiness'],
                'tempo': audio_features[track_id]['tempo'],
                'time_signature': audio_features[track_id]['time_signature'],
                'valence': audio_features[track_id]['valence']
            })
        else:
            # オーディオ特徴が取得できなかった場合
            track.update({
                'acousticness': None,
                'danceability': None,
                'energy': None,
                'instrumentalness': None,
                'key': None,
                'liveness': None,
                'loudness': None,
                'mode': None,
                'speechiness': None,
                'tempo': None,
                'time_signature': None,
                'valence': None
            })
    return track_info_list


def main(query, desired_total):
    try:
        track_info_list = search_tracks(query, limit=desired_total)
        # トラックIDのリストを作成
        track_ids = [track['id'] for track in track_info_list if track['id']]

        # オーディオ特徴の取得（100件まで一度に取得可能）
        audio_features = get_audio_features(track_ids)
        # 曲情報とオーディオ特徴の統合
        track_info_list = merge_track_info_with_audio_features(track_info_list, audio_features)
        # 結果の表示
        display_tracks(track_info_list)
        return track_info_list
    except spotipy.exceptions.SpotifyException as e:
        print(f"Spotify APIエラー: {e}")
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")

if __name__ == "__main__":
    query_word = 'track:"喜び"' # 検索キーワード
    #query_word = 'artist:"Kenshi" genre:"j-pop"'
    num_music = 100  # 取得したい総件数
    result = main(query_word, num_music)


見つかった曲の数: 100

曲 1:
  id: 2ymeOsYijJz09LfKw3yM2x
  name: Joy To The World
  artists: Three Dog Night
  release_date: 1970-01-01
  danceability: 0.649
  valence: 0.971
曲 2:
  id: 1VZ04sZSpTet7wVMyZphA8
  name: Joy To The World
  artists: Three Dog Night
  release_date: 2003-01-01
  danceability: 0.591
  valence: 0.968
曲 3:
  id: 4VTnc3rElZ3Ihcz5VM8mXa
  name: L'isle joyeuse, L. 106
  artists: Claude Debussy, Seong-Jin Cho
  release_date: 2017-11-17
  danceability: 0.289
  valence: 0.0324
曲 4:
  id: 79e2s6puD7ZVF9DZywh6Hy
  name: Big好き (suki)
  artists: &TEAM
  release_date: 2024-10-22
  danceability: 0.9
  valence: 0.859
曲 5:
  id: 1aLFjQNgm5HxLqpErPCcYR
  name: 喜びのうた ～Fun in da Sound～
  artists: Manji Line
  release_date: 2014-07-02
  danceability: 0.724
  valence: 0.618
曲 6:
  id: 1nDG5R1vENQwAv0q1Rnqg1
  name: 喜びのかけら
  artists: Kumi Koda
  release_date: 2017-03-08
  danceability: 0.49
  valence: 0.395
曲 7:
  id: 2yaWLvSQJ7lArJrRvGvc8C
  name: 喜びも悲しみも幾歳月
  artists: 若山彰
  release_date: